In [1]:
import numpy as np
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout

Using TensorFlow backend.


In [2]:
def load_data(filename, skiprows = 1):
    """
    Function loads data stored in the file filename and returns it as a numpy ndarray.
    
    Inputs:
        filename: given as a string.
        
    Outputs:
        Data contained in the file, returned as a numpy ndarray
    """
    return np.loadtxt(filename, skiprows=skiprows, delimiter=' ')

In [3]:
# Loads training dataset
training = load_data('data/training_data.txt', 1)
X_train = training[:, 1:]
Y_train = training[:,0]

training shape: (20000, 1001)


In [4]:
# Loads testing dataset
# There is no label for testing set 
X_test = load_data('data/test_data.txt', 1)

In [5]:
# Converting Y values to one hot vector
num_classes = 2
Y_train = keras.utils.to_categorical(Y_train, num_classes=num_classes)


Y_train shape: {} (20000,)
Y_train[0] shape: {} ()


In [6]:
# Input size
n_train = X_train.shape[0]
n_words = X_train.shape[1]
n_test = Y_train.shape[0]


X_train shape: {} (20000, 1000)
X_train[0] shape: {} (1000,)
Y_train shape: {} (20000, 2)
Y_train[0] shape: {} (2,)


In [8]:
## Create your own model here given the constraints in the problem
model = Sequential()
model.add(Dense(100, input_shape=(n_words,)))
model.add(Activation('relu'))



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               100100    
_________________________________________________________________
activation_3 (Activation)    (None, 100)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 202       
_________________________________________________________________
activation_4 (Activation)    (None, 2)                 0         
Total params: 100,302
Trainable params: 100,302
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
20000/20000 [==============================] - 1s 56us/step - loss: 0.4261 - acc: 0.8146
Epoch 2/10
20000/20000 [============

In [20]:
prediction = model.predict(X_test, verbose=1)
print(prediction)

zeros = prediction[:, 0]
ones  = prediction[:, 1]

prediction = (zeros < ones).astype(int)

print(prediction)
print("prediction shape is: {}".format(prediction.shape))

output = "Id,Prediction\n"
for i in range(prediction.shape[0]):
    output = output + ("{0},{1}\n".format(i + 1, prediction[i]))

file = open('predictions.csv','w') 
file.write(output)
file.close()


10000/10000 [==============================] - 0s 36us/step
[[0.07278527 0.92721474]
 [0.05613114 0.9438689 ]
 [0.9128117  0.0871883 ]
 ...
 [0.99837637 0.00162358]
 [0.00460773 0.9953923 ]
 [0.9785726  0.02142744]]
[1 1 0 ... 0 1 0]
prediction shape is: (10000,)
